<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>
<h2 align=center><font size = 3>Notebook focusing on data cleansing</font></h2>


## Overview

This is the first notebook created to extract and cleanse the data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

Instead of using BeautifulSoup which requires longer learning curve, I used shell scripts in Linux to extract the required data and converted the html source code into a csv file. 

The following highlights the steps I took:

#### Step 1:  
get the html source of the above page from view-source:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, save all in a text file named ```toronto_data.txt```.

#### Step 2:
remove all the lines before and after the table of interest (i.e., only the lines between <table ...> at line 60 and </table> at line 1508 are kept.)

#### Step 3:
execute the following shell command to get the table elements and strip the HTML tags 
```
> cat toronto_data.txt |egrep "<td>" |sed "s,</td>,," |sed "s,</a>,," |sed "s,.*\>,," > list

```
The result is saved in the text in a file named ```list```.

#### Step 4:
run a script to convert the single list into a three column csv file:

``` 
>./script > toronto_data.csv

```
The script has the following content:
```
    i=1;
    new_line=""
    while read line; do
          r=`expr $i % 3`
          #remove "\n" from the line
          word=`echo $line |sed "s/\n//"`
          new_line="${new_line}$word" 
          if [ $r != 0 ];
          then
            new_line="$new_line,";
          else
            echo "$new_line";
            new_line="";
          fi
          i=`expr $i + 1` 
    done < "list"

```
The cvs file looks like the following:

```
    M1A,Not assigned,Not assigned 
    M2A,Not assigned,Not assigned 
    M3A,North York,Parkwoods 
    M4A,North York,Victoria Village 
    M5A,Downtown Toronto,Harbourfront 
    M5A,Downtown Toronto,Regent Park 
    M6A,North York,Lawrence Heights 
... ...   
```


In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


<a id='item1'></a>

In [2]:
# read the csv file obtained as described above
df = pd.read_csv("toronto_data.csv")

In [3]:
print("dimension of data:",df.shape)

dimension of data: (288, 3)


In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# remove entries with "Borough" as "Not assigned"
df = df[df['Borough'] != 'Not assigned']
print("dimention of data:", df.shape)
df.head()

dimention of data: (211, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [6]:
# fill in the Neighboorhoods that are "Not assigned" with the "Borough" value

ind = df[df['Neighborhood']=='Not assigned'].index.tolist()
df_borough = df.loc[ind,:]['Borough']
df.loc[ind,'Neighborhood'] = df_borough


In [7]:
# make sure there is no more Neighorhood with "Not Assigned"
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [8]:
# Add "," to end of each Neighborhood name before aggregating by PostalCode and Borough
df_marked = df['Neighborhood']+', '
new_df = df.copy()
new_df['Neighborhood'] = df_marked
df_grouped = new_df.groupby(['PostalCode','Borough']).sum()
df_grouped.head()


,,Neighborhood
PostalCode,Borough,
M1B,Scarborough,"Rouge, Malvern,"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union,"
M1E,Scarborough,"Guildwood, Morningside, West Hill,"
M1G,Scarborough,"Woburn,"
M1H,Scarborough,"Cedarbrae,"


In [9]:
# flatten the dataframe
toronto_data = df_grouped.reset_index()
print("dimentions of data", toronto_data.shape)
toronto_data.head()

dimentions of data (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern,"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union,"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,"
3,M1G,Scarborough,"Woburn,"
4,M1H,Scarborough,"Cedarbrae,"


In [10]:
# remove the trailing "," in "Neighborhood"
df_unmarked =toronto_data['Neighborhood'].str.replace(", $","")
df_unmarked

toronto_data['Neighborhood']=df_unmarked
print("dimentions of data", toronto_data.shape)
toronto_data.head()

dimentions of data (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


At this point, the data frame "toronto_data" is read for use. writing this to the csv so the next notebook can read it in for further process.

In [11]:
toronto_data.set_index("PostalCode").to_csv("toronto_data_clean.csv")